# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [ ]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

In [ ]:
import sys
sys.path.append('../../../')

import os
import sys

In [ ]:
import sagemaker
import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [ ]:
from deep.constants import *
from deep.utils import *

In [ ]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [ ]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

train_val_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_train_val.csv')).drop_duplicates()
#tot_df = tot_df[tot_df]

test_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_test_v0.7.1.csv'))[['excerpt']]

In [ ]:
from ast import literal_eval
def flatten(t):
    return [item for sublist in t for item in sublist]

len(list(set(flatten(train_val_df['target'].apply(literal_eval)))))

In [ ]:
columns = ['excerpt', 'entry_id', 'target']
train_val_df = train_val_df[columns]

In [ ]:
"""tot_df = pd.concat(
    [
        tot_df,
        pd.read_csv(os.path.join(DATA_PATH, 'augmented_subpillars.csv'))
    ]
)"""

In [ ]:
train_val_df.columns

In [ ]:
train_val_df.shape

## Sagemaker Prep

### Session

Configure SageMaker

In [ ]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [ ]:
MLFLOW_SERVER

In [ ]:
sample = True  # To make the computations faster, sample = True.

if sample:
    tot_df = train_val_df.sample(n=20_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')

tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [ ]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

# CPU instances
instances = [
    'ml.c4.2xlarge',
    'ml.c4.4xlarge',
    'ml.c5n.2xlarge'
]

# https://aws.amazon.com/sagemaker/pricing/instance-types

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [ ]:
from sagemaker.pytorch import PyTorch

instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-new-architecture",
    'max_len': 128,
    'epochs': 1,
    'model_name': "microsoft/xtremedistil-l6-h384-uncased",
    'tokenizer_name': "microsoft/xtremedistil-l6-h384-uncased",
    'output_length': 384,
    'learning_rate': 3e-5,
    "instance_type": instance_type,
    'beta_f1': 0.7,
    'nb_repetitions': 1,
    'run_name': 'all_tags_final',
    'train_batch_size': 32
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str(
        '../../../scripts/training/selim/multiclass-lightning/MultitaskAllInOne'
    ),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py3",
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [ ]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [ ]:
# Fit the estimator
estimator.fit(fit_arguments, job_name=job_name)